# Mocule for Molecular Dynamics.

In [9]:
#!/bin/env python
import os
import sys
import time


In [19]:
class LigParGen():
    def __init__(self, is_headless=True):

        self.url = 'http://zarbi.chem.yale.edu/ligpargen/'
        self.prefs = {"download.default_directory" : os.getcwd()}


    def get_opls_from_smiles(self,
            smiles='CCC',
            format='lammps',
            to='filename',
            is_headless=True,
    ):
        import selenium
        from selenium import webdriver
        from selenium.webdriver.common.keys import Keys
        self.options = webdriver.ChromeOptions()
    #    options.add_argument('--headless')
        self.options.add_experimental_option('prefs', self.prefs)
        if is_headless: 
            self.options.add_argument('--headless')
        self.driver = webdriver.Chrome(options=self.options)
        
        self.driver.get(self.url)
        
        print(self.driver.title)
        box = self.driver.find_element_by_id('smiles')
        print(box)
        box.send_keys(smiles)

        time.sleep(10)

        b = self.driver.find_element_by_class_name("btn")
        print(b)
        b.submit()

        time.sleep(10)

        p = self.driver.find_elements_by_css_selector('p')
        print(p)
        f = self.driver.find_element_by_class_name('form-group')
        keys = [_.get_property('value') for _ in  f.find_elements_by_name('go')]
        d = dict(list(zip(keys, f.find_elements_by_name('go'))))
        d['LAMMPS'].submit()

        time.sleep(10)

    def format_lammpsdata(self):
        pass

In [20]:
def test():
    l = LigParGen()
    from pathlib import Path
    savedir = '../../ligpargen_data'
    path = Path(savedir).resolve()
    cwd = Path('.').resolve()
    if path.exists():
        with (path / '0.data').open() as f:
            lines = f.read()
    else:
        l.get_opls_from_smiles('C')
        
    return lines

In [40]:

from io import StringIO
from ase.io import read
l = LigParGen()
savedir = '../../ligpargen_data'
path = Path(savedir).resolve()
cwd = Path('.').resolve()
if path.exists():
    with (path / '0.data').open() as f:
        lines = f.read()
with StringIO() as f:
    f.write(lines)
    f.seek(0)
    a = read(f, format='lammps-data')
print(a)
charges = a.get_initial_charges()
masses = a.get_masses()
cell = a.get_cell()
positions = a.get_positions()
bonds = a.arrays['bonds']
angles = a.arrays['angles']
dihedrals = a.arrays['dihedrals']
atomids = a.arrays['id']
molids = a.arrays['mol-id']
types = a.arrays['type']
mmcharges = a.arrays['mmcharges']
print(a.arrays.keys())
a.arrays['bonds']

Atoms(symbols='HHeLiBeBCNOFNeNaMgAlSiPSClArKCaScTiVCrMnFeCoNiCuZnGaGeAsSeBrKr', pbc=True, cell=[50.0, 50.0, 50.0], angles=..., bonds=..., dihedrals=..., id=..., initial_charges=..., masses=..., mmcharges=..., mol-id=..., type=...)
dict_keys(['numbers', 'positions', 'masses', 'id', 'type', 'mol-id', 'initial_charges', 'mmcharges', 'bonds', 'angles', 'dihedrals'])


array(['_', '0(1)', '1(2)', '2(3)', '3(4)', '4(5)', '5(6)', '6(7)',
       '6(8)', '8(9)', '9(10)', '10(11)', '11(12)', '12(13)', '0(14)',
       '0(15)', '0(16)', '1(17)', '1(18)', '2(19)', '2(20)', '3(21)',
       '3(22)', '4(23)', '4(24)', '9(25)', '9(26)', '10(27)', '10(28)',
       '11(29)', '11(30)', '12(31)', '12(32)', '13(33)', '13(34)',
       '13(35)'], dtype='<U6')

In [50]:
print([_ for _ in dir(a) if not _.startswith('__')])

['_ase_handles_dynamic_stress', '_calc', '_celldisp', '_cellobj', '_centering_as_array', '_constraints', '_del_constraints', '_get_atomic_numbers', '_get_constraints', '_get_positions', '_masked_rotate', '_pbc', '_set_positions', 'append', 'arrays', 'ase_objtype', 'calc', 'cell', 'center', 'constraints', 'copy', 'edit', 'euler_rotate', 'extend', 'fromdict', 'get_all_distances', 'get_angle', 'get_angles', 'get_angular_momentum', 'get_array', 'get_atomic_numbers', 'get_calculator', 'get_cell', 'get_cell_lengths_and_angles', 'get_celldisp', 'get_center_of_mass', 'get_charges', 'get_chemical_formula', 'get_chemical_symbols', 'get_dihedral', 'get_dihedrals', 'get_dipole_moment', 'get_distance', 'get_distances', 'get_forces', 'get_global_number_of_atoms', 'get_initial_charges', 'get_initial_magnetic_moments', 'get_kinetic_energy', 'get_magnetic_moment', 'get_magnetic_moments', 'get_masses', 'get_momenta', 'get_moments_of_inertia', 'get_number_of_atoms', 'get_pbc', 'get_positions', 'get_poten

In [246]:
body = lines.split('\n\n')
header_tags = ['atoms',
 'bonds',
 'angles',
 'dihedrals',
 'impropers']
header_tags_types = ['atom types', 'bond types', 'angle types', 'dihedral types', 'improper types']
tags_atomic = ['Masses',
 'Atoms', 'Velocities']

tags_interactions = [
 'Bonds',
 'Angles',
 'Dihedrals',
 'Impropers']

tags_coeffs = ['Pair Coeffs', 'Nonbond Coeffs',
 'Bond Coeffs',
 'Angle Coeffs',
 'Dihedral Coeffs',
 'Improper Coeffs',
 'BondBond Coeffs',
 'BondAngle Coeffs',
 'MiddleBondTorsion Coeffs',
 'EndBondTorsion Coeffs',
 'AngleTorsion Coeffs',
 'AngleAngleTorsion Coeffs',
 'BondBond13 Coeffs',
 'AngleAngle Coeffs']

tmp = """
Sample file with Annotations

Here is a sample file with annotations in parenthesis and lengthy sections replaced by dots (...). Note that the blank lines are important in this example.


LAMMPS Description           (1st line of file)

100 atoms         (this must be the 3rd line, 1st 2 lines are ignored)
95 bonds                (# of bonds to be simulated)
50 angles               (include these lines even if number = 0)
30 dihedrals
20 impropers

5 atom types           (# of nonbond atom types)
10 bond types          (# of bond types = sets of bond coefficients)
18 angle types         
20 dihedral types      (do not include a bond,angle,dihedral,improper type
2 improper types             line if number of bonds,angles,etc is 0)

-0.5 0.5 xlo xhi       (for periodic systems this is box size,
-0.5 0.5 ylo yhi        for non-periodic it is min/max extent of atoms)
-0.5 0.5 zlo zhi       (do not include this line for 2-d simulations)

Masses

  1 mass
  ...
  N mass                           (N = # of atom types)

Nonbond Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of atom types)

Bond Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of bond types)

Angle Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of angle types)

Dihedral Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of dihedral types)

Improper Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of improper types)

BondBond Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of angle types)

BondAngle Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of angle types)

MiddleBondTorsion Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of dihedral types)

EndBondTorsion Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of dihedral types)

AngleTorsion Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of dihedral types)

AngleAngleTorsion Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of dihedral types)

BondBond13 Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of dihedral types)

AngleAngle Coeffs

  1 coeff1 coeff2 ...
  ...
  N coeff1 coeff2 ...              (N = # of improper types)

Atoms

  1 molecule-tag atom-type q x y z nx ny nz  (nx,ny,nz are optional -
  ...                                    see "true flag" input command)
  ...                
  N molecule-tag atom-type q x y z nx ny nz  (N = # of atoms)

Velocities

  1 vx vy vz
  ...
  ...                
  N vx vy vz                        (N = # of atoms)

Bonds

  1 bond-type atom-1 atom-2
  ...
  N bond-type atom-1 atom-2         (N = # of bonds)

Angles

  1 angle-type atom-1 atom-2 atom-3  (atom-2 is the center atom in angle)
  ...
  N angle-type atom-1 atom-2 atom-3  (N = # of angles)

Dihedrals

  1 dihedral-type atom-1 atom-2 atom-3 atom-4  (atoms 2-3 form central bond)
  ...
  N dihedral-type atom-1 atom-2 atom-3 atom-4  (N = # of dihedrals)

Impropers

  1 improper-type atom-1 atom-2 atom-3 atom-4  (atom-2 is central atom)
  ...
  N improper-type atom-1 atom-2 atom-3 atom-4  (N = # of impropers)
  """

In [247]:
[_.split('  ')[0].split(None, 1)[1].split()[0] for _ in tmp.split('\n') if ' types ' in _.lower()]

['atom', 'bond', 'angle', 'dihedral', 'improper']

In [248]:
import re

tags_all = tags_atomic + tags_interactions + tags_coeffs
{_tag:re.findall(f'{_tag}.*?\n\n(.*?)\n\n', lines, re.DOTALL) for _tag in tags_all}

{'Masses': [],
 'Atoms': [],
 'Velocities': [],
 'Bonds': [],
 'Angles': [],
 'Dihedrals': [],
 'Impropers': [],
 'Pair Coeffs': [],
 'Nonbond Coeffs': [],
 'Bond Coeffs': [],
 'Angle Coeffs': [],
 'Dihedral Coeffs': [],
 'Improper Coeffs': [],
 'BondBond Coeffs': [],
 'BondAngle Coeffs': [],
 'MiddleBondTorsion Coeffs': [],
 'EndBondTorsion Coeffs': [],
 'AngleTorsion Coeffs': [],
 'AngleAngleTorsion Coeffs': [],
 'BondBond13 Coeffs': [],
 'AngleAngle Coeffs': []}

In [285]:

def read_lammpsdata(filename):
    import numpy as np
    import pandas as pd
    if os.path.exists(filename):
        with open(filename) as f:
            lines = f.read()
    else:
        lines = filename

    data = dict()

    tags = tags_atomic + tags_interactions + tags_coeffs
    for _tag in tags:
        p = f'(^{_tag}\s*\n\n)(.*?)\n\n'
        match = re.split(p, lines, flags=re.MULTILINE|re.DOTALL)
        print(_tag, len(match))
        if len(match) == 4:
            _prev, _, _match, _next = match
            _lines = re.findall('^.*$', _match, flags=re.MULTILINE)
            _lines = [_line.split('#') for _line in _lines]
            _comments = ['' if len(_line)==1 else _line[1] for _line in _lines]
            _data = [_line[0].split() for _line in _lines]
            if _tag in tags_interactions:
                _columns = ['id', 'type', *[f'i{_}' for _ in range(len(_data[0])-2)]]
                _data = pd.DataFrame(np.array(_data), dtype=int, columns=_columns)
                _data['comment'] = _comments
            elif _tag in tags_coeffs:
                _columns
                _data = pd.DataFrame(_data)
            elif _tag == 'Masses':
                _data = pd.DataFrame(_data, columns=['atomid', 'mass'])
                _data['atomid'] = _data['atomid'].astype(int)
                _data['mass'] = _data['mass'].astype(float)
                _data['comment'] = _comments
            elif _tag == 'Velocity':
                _data = pd.DataFrame(np.array(_data, dtype=float), columns=['atomid', 'x', 'y', 'z'])
                _data['atomid'] = data['atomid'].astype(int)
                _data['comment'] = _comments
            data[_tag] = _data

            lines = _prev + _next
    
    title, lines = lines.split('\n', 1)
    data['title'] = title

    tags = header_tags_types + header_tags
    for _tag in tags:
        p = f'^([-+.\d\s]+)({_tag}.*?)$'
        match = re.split(p, lines, flags=re.MULTILINE)
        print(_tag, len(match))
        if len(match) == 4:
            _prev, _value, _match, _next = match
            _line = _match.split('#')
            _comment = '' if len(_line)==1 else _line[1]
            data[_tag] = int(_value)
            
            lines = _prev + _next
    
    _lines = lines.strip().splitlines()
    lines = []
    for _line in _lines:
        if 'lo' in _line and 'hi' in _line:
            _low, _high, _tag_low, _tag_high = _line.strip().split(None, 3)
            _tag_high = _tag_high.split()[0]
            data[_tag_low] = float(_low)
            data[_tag_high] = float(_high)
        else:
            lines.append(_line)
    lines = '\n'.join(lines)

    return data, lines

data, lines = read_lammpsdata('../../ligpargen_data/0.data')

Masses 4
Atoms 4
Velocities 1
Bonds 4
Angles 4
Dihedrals 4
Impropers 4
Pair Coeffs 4
Nonbond Coeffs 1
Bond Coeffs 4
Angle Coeffs 4
Dihedral Coeffs 4
Improper Coeffs 4
BondBond Coeffs 1
BondAngle Coeffs 1
MiddleBondTorsion Coeffs 1
EndBondTorsion Coeffs 1
AngleTorsion Coeffs 1
AngleAngleTorsion Coeffs 1
BondBond13 Coeffs 1
AngleAngle Coeffs 1
atom types 4
bond types 4
angle types 4
dihedral types 4
improper types 4
atoms 4
bonds 4
angles 4
dihedrals 4
impropers 4


In [286]:
lines.strip()

''

In [287]:
for k,v in data.items():
    if np.isscalar(v):
        print(k,v)
    elif isinstance(v, pd.DataFrame):
        print(k, v.iloc[0])
    else:
        print(k, v[0])

Masses atomid          1
mass       12.011
comment          
Name: 0, dtype: object
Atoms ['1', '1', '1', '-0.23820000', '1.000', '1.00000', '0.00000']
Bonds id         1
type       1
i0         2
i1         1
comment     
Name: 0, dtype: object
Angles id         1
type       1
i0         1
i1         2
i2         3
comment     
Name: 0, dtype: object
Dihedrals id         1
type       1
i0         4
i1         3
i2         2
i3         1
comment     
Name: 0, dtype: object
Impropers id         1
type       1
i0         7
i1         9
i2         6
i3         8
comment     
Name: 0, dtype: object
Pair Coeffs ['1', '0.066', '3.5000000']
Bond Coeffs ['1', '268.0000', '1.5290']
Angle Coeffs ['1', '58.350', '112.700']
Dihedral Coeffs ['1', '1.300', '-0.200', '0.200', '0.000']
Improper Coeffs ['1', '0.000', '-1', '2']
title LAMMPS data file Created by LigParGen - (Written by Leela S. Dodda)
atom types 36
bond types 35
angle types 65
dihedral types 82
improper types 21
atoms 36
bonds 35
angles